In [ ]:
!git clone https://github.com/lsewcx/Pytorch-UNet.git

In [ ]:
!pip install segmentation_models_pytorch
!pip install -U albumentations

In [25]:
import os
import numpy as np
from PIL import Image
import albumentations as A
from tqdm import tqdm

# 定义数据增强操作
augmentation = A.Compose([
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.OneOf([
        A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=None, p=0.5),
        A.GridDistortion(p=0.5),
        A.OpticalDistortion(distort_limit=2, shift_limit=0.5, p=1)
    ], p=0.8),
    A.Resize(224, 224),
    A.RandomBrightnessContrast(p=0.8),
    A.RandomGamma(p=0.8)
], is_check_shapes=False)

# 定义文件夹路径
images_dir = '/kaggle/working/Pytorch-UNet/NEU_Seg-main/images/training'
masks_dir = '/kaggle/working/Pytorch-UNet/NEU_Seg-main/annotations/training'
augmented_images_dir = '/kaggle/working/Pytorch-UNet/NEU_Seg-main/images/training'
augmented_masks_dir = '/kaggle/working/Pytorch-UNet/NEU_Seg-main/annotations/training'

# 创建保存增强样本的文件夹
os.makedirs(augmented_images_dir, exist_ok=True)
os.makedirs(augmented_masks_dir, exist_ok=True)

# 获取所有图像文件名
image_files = [f for f in os.listdir(images_dir) if os.path.isfile(os.path.join(images_dir, f))]

# 对每个图像应用数据增强并保存
for image_file in tqdm(image_files):
    # 加载图像和掩码
    img = np.array(Image.open(os.path.join(images_dir, image_file)))
    mask = np.array(Image.open(os.path.join(masks_dir, image_file.replace('.jpg', '.png'))))

    # 应用数据增强
    augmented = augmentation(image=img, mask=mask)
    augmented_img = augmented['image']
    augmented_mask = augmented['mask']

    # 生成新的文件名
    base_name, ext = os.path.splitext(image_file)
    augmented_image_file = f"{base_name}_aug{ext}"
    augmented_mask_file = f"{base_name}_aug_mask.png"

    # 保存增强后的图像和掩码
    Image.fromarray(augmented_img).save(os.path.join(augmented_images_dir, augmented_image_file))
    Image.fromarray(augmented_mask).save(os.path.join(augmented_masks_dir, augmented_mask_file))

# 输出文件夹中的样本数量
augmented_image_files = [f for f in os.listdir(augmented_images_dir) if os.path.isfile(os.path.join(augmented_images_dir, f))]
augmented_mask_files = [f for f in os.listdir(augmented_masks_dir) if os.path.isfile(os.path.join(augmented_masks_dir, f))]

print(f'增强后的图像数量: {len(augmented_image_files)}')
print(f'增强后的掩码数量: {len(augmented_mask_files)}')

100%|██████████| 4170/4170 [00:27<00:00, 151.41it/s]


增强后的图像数量: 8340
增强后的掩码数量: 8340


In [ ]:
%cd Pytorch-UNet
!rm -rf NEU_Seg-main
!unzip -q ceping/NEU_Seg-main.zip

In [ ]:
!mv /kaggle/working/Pytorch-UNet/NEU_Seg-main/images/test/* /kaggle/working/Pytorch-UNet/NEU_Seg-main/images/training/
!mv /kaggle/working/Pytorch-UNet/NEU_Seg-main/annotations/test/* /kaggle/working/Pytorch-UNet/NEU_Seg-main/annotations/training/

In [ ]:
%cd /kaggle/working/Pytorch-UNet
!git pull
!rm -rf /kaggle/working/Pytorch-UNet/model
!rm -rf best_model.pth best_model.pt
!python unet_train.py --classes 4 --batch-size 16 --epochs 100 --scale 1 --model UNet --v 20 -l 0.0001

/kaggle/working/Pytorch-UNet
Already up to date.


In [ ]:
%cd /kaggle/working/Pytorch-UNet
!git pull
!python unet_predict.py --model best_model.pth --scale 1 --model-name UNet -i NEU_Seg-main/images/test --batch-size 32

In [ ]:
!pip install segmentation-models-pytorch

In [ ]:
%cd Pytorch-UNet
!git pull

In [ ]:
%cd Pytorch-UNet
!rm -rf best_model.pth best_model.pt
!python train.py --classes 4 --batch-size 16 --epochs 100 --scale 1 --model selfnet --v 20

In [ ]:
import torch
import segmentation_models_pytorch as smp

model = torch.load('/kaggle/working/Pytorch-UNet/ceping/best_model.pth')

print(model)

In [ ]:
# !rm -rf archive_name.zip /kaggle/working/Pytorch-UNet/test_predictions 
# %cd /kaggle/working/Pytorch-UNet
# !mv best_model.pth ceping/best_model.pth
# !python predict.py --model  /kaggle/working/Pytorch-UNet/ceping/best_model.pth --scale 1 --model-name selfnet -i NEU_Seg-main/images/test
!rm -rf ceping/test_predictions
!mv test_predictions ceping/test_predictions

In [ ]:
!rm -rf ceping/baseline_predictions
!mv test_predictions ceping/baseline_predictions

In [ ]:
%cd ceping
!python main.py
%cd ..

In [ ]:
%cd /kaggle/working/Pytorch-UNet
!rm -rf submsion.zip
!zip submsion.zip ceping/best_model.pth ceping/results.json  ceping/results.txt ceping/test_predictions/*

In [ ]:
%cd /kaggle/working
!rm -rf predictions.zip
!zip predictions.zip /kaggle/working/Pytorch-UNet/test_predictions/*

In [ ]:
import os
os.chdir('/kaggle/working')
print(os.getcwd())
print(os.listdir("/kaggle/working"))
from IPython.display import FileLink
FileLink("predictions.zip")
# FileLink('Pytorch-UNet/submsion.zip')

In [ ]:
!pip install segmentation-models-pytorch